In [1]:
from load_data import df
from distance_matrix import add_directions_columns
import pandas as pd

/usr/local/Caskroom/miniconda/base/envs/property-data/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [66]:
HIDE_USE_CODES = [
    "101",
    "132",
    "130",
    "104",
    "111",
    "105",
    "112",
    "109",
    "900",
    "931",
    "934"
]

In [71]:
filtered = df.assign(
    distance=df.geom.distance(list(df[df.SITE_ADDR == "332 MAIN ST"].geom)[0].centroid)
).query(
    'CITY.str.lower() == "greenfield" and YEAR_BUILT <= 1930 and USE_CODE != @HIDE_USE_CODES and BLD_AREA > 6000 and distance < 600'
)

In [82]:
filtered.explore(   
    column="FULL_STR",
    # tiles="USGS.USImagery",
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='ESRI',
    popup=True,
    style_kwds={"fillOpacity": 0.2}
).save(
    'wharehouse.html'
)

In [73]:

with pd.option_context('display.max_rows', None):
    print(filtered.agg('{0[USE_CODE]}: {0[USE_DESC]}'.format, axis=1).value_counts())

031: Mixed Use (Primarily Commercial, some Residential)                                                      18
325: Small Retail and Services stores (under 10,000 sq. ft.)                                                 18
013: Mixed Use (Primarily Residential, some Commercial)                                                      15
340: General Office Buildings                                                                                14
960: Church, Mosque, Synagogue, Temple, etc...                                                                7
316: Other Storage, Warehouse, and Distribution facilities (see also usecode 401)                             5
400: Buildings for manufacturing operations                                                                   4
326: Eating and Drinking Establishments - restaurants, diners, fast food establishments, bars, nightclubs     4
957: Charitable Services                                                                                

In [81]:
filtered.head()

,BLDG_VAL,LAND_VAL,OTHER_VAL,TOTAL_VAL,FY,LOT_SIZE,LS_DATE,LS_PRICE,USE_CODE,SITE_ADDR,ADDR_NUM,FULL_STR,LOCATION,CITY,ZIP,OWNER1,OWN_ADDR,OWN_CITY,OWN_STATE,OWN_ZIP,OWN_CO,LS_BOOK,LS_PAGE,REG_ID,ZONING,YEAR_BUILT,BLD_AREA,UNITS,RES_AREA,STYLE,STORIES,NUM_ROOMS,LOT_UNITS,CAMA_ID,USE_DESC,rowid,geom,distance
2281,274000,48000,6500,328500,2022,0.45,20050401,150000,013,111 HOPE ST,111,HOPE ST,,GREENFIELD,01301,FENWICK LLP,PO BOX 367,DEERFIELD,MA,01342,,4821,272,,RA,1900,13170,6,10134,APRTMNT-GN,3,12,A,1607,"Mixed Use (Primarily Residential, some Commerc...",M_109879_926420,"POLYGON ((109909.171 926410.903, 109909.495 92...",575.715124
2430,403100,53900,4300,461300,2022,0.35,20020419,150700,013,44 46 DEERFIELD ST,44 46,DEERFIELD ST,,GREENFIELD,01301,CHASE REALTY TRUST,18 CHAPMAN STREET,GREENFIELD,MA,01301,,3988,305,,GC,1890,13904,6,9176,COMM BLOCK,3,14,A,1742,"Mixed Use (Primarily Residential, some Commerc...",M_109622_926536,"POLYGON ((109654.745 926529.835, 109600.379 92...",432.033888
2435,107300,61000,2300,170600,2022,0.65,20000127,50000,400,106 HOPE ST,106,HOPE ST,,GREENFIELD,01301,DUDA REALTY LLC,P O BOX 1531,GREENFIELD,MA,01302,,3591,16,,,1890,31922,0,16320,MILL,2,0,A,1747,Buildings for manufacturing operations,M_109827_926407,"POLYGON ((109861.229 926360.538, 109841.354 92...",544.095035
2446,319400,63900,5800,389100,2022,0.73,19990525,315000,331,18 DEERFIELD ST,18,DEERFIELD ST,,GREENFIELD,01301,FRANKLIN REALTY LLC,120 NORTH COLONY ROAD,WAILINGFORD,CT,06492,US,3498,100,,GC,1930,13297,0,12575,WAREHOUSE,1,0,A,1757,Automotive Supplies Sales and Service,M_109594_926593,"POLYGON ((109626.585 926607.229, 109609.153 92...",357.208269
2448,173900,52600,7100,233600,2022,0.30,20150109,275000,031,10 MILL ST,10,MILL ST,,GREENFIELD,01301,NICKATE PROPERTY LLC,555 MAIN STREET,KEENE,NH,03431,,6635,45,,GC,1915,7088,0,4935,STORE,1,0,A,1759,"Mixed Use (Primarily Commercial, some Resident...",M_109563_926616,"POLYGON ((109573.048 926641.658, 109574.555 92...",362.960014


In [79]:
filtered.to_csv('wharehouse.csv')